In [2]:
### 100,000 UK Used Car Data set
https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes


1. Выбрать набор данных для своей группы: <br>
    3530203_70301: Mercedes<br>
    3530203_70302: Ford <br>
    3530903_70101: BMW <br>
    3530903_70102: Audi <br>
    
2. Разбить данные на тренировочные и тестовые (70% на 30%, random_state=10).
3. Применить доступные способы предобработки данных.
4. Попытаться обучить регрессию для предсказания цены автомобиля c наилучшим качеством. Качество оценивать по метрике RMSE на тестовой подвыборке. Попробовать различные модели регуляризации (и не только).

SyntaxError: invalid syntax (Temp/ipykernel_9304/746141072.py, line 2)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
np.set_printoptions(precision=3)
pd.set_option('precision', 3)

sns.set()

In [ ]:
auto_data = pd.read_csv("merc.csv")
auto_data

# Exploring Data and Preprocessing Data

In [ ]:
auto_data.info()

In [ ]:
auto_data.describe()

In [ ]:
auto_data.sort_values('mileage', ascending=True)

In [ ]:
auto_data.sort_values('mpg', ascending=True)

**У модели A class mpg не может быть равен 1.1, поэтому удалим их из датасета**

In [ ]:
auto_data.loc[:,['mpg']] = auto_data['mpg'].replace(to_replace=[1.1],value=np.NaN)
auto_data.dropna(how='any', axis=0, inplace=True)

Значение mpg = 217 не выглядит, как выброс

In [ ]:
df = auto_data.model.isin(['A Class'])
auto_data[df]

**Не работает. Почему?**

In [ ]:

plt.figure(figsize=(25,8))
sns.countplot(x = 'model',
              data = auto_data,
              order = auto_data['model'].value_counts().index)

plt.show()

**Проверим на null значения**

In [ ]:
auto_data.isnull().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.subplots(figsize=(12,7))
sns.barplot(x=auto_data['year'], y=auto_data['price'])

In [ ]:
year_1970_df = auto_data['year'] == 1970
auto_data[year_1970_df]

Уберем 1970 год со слишком высокой ценой из датафрейма, так как он может испортить модель

In [ ]:
auto_data.drop(labels = [12072],axis = 0, inplace=True)

In [ ]:
auto_data.sort_values(by='price', ascending=False)

In [ ]:
auto_data.sort_values(by='price')

Price 650 выглядит как выброс, лучше выбросить

In [ ]:
auto_data.drop(labels = [11816],axis = 0, inplace=True)

In [ ]:
auto_data.sort_values(by='price')

**Заменим год выпуска автомобиля на ее возраст**

In [ ]:
auto_data['year'] = 2020-auto_data['year']

In [ ]:
plt.subplots(figsize=(12,7))
sns.barplot(x=auto_data['year'], y=auto_data['price'])

In [ ]:
plt.subplots(figsize=(12,7))
sns.countplot(auto_data['year'])

In [ ]:
df = auto_data.year == 9
auto_data[df].sort_values(by = 'price')

Похоже на правду

In [ ]:
df = auto_data.year == 22
auto_data[df].sort_values(by = 'price')

SLK 1998 года за 1990, почему так дешево?

In [ ]:
#auto_data.drop(labels = [11764],axis = 0, inplace=True)
#auto_data.drop(labels = [11808],axis = 0, inplace=True)

In [ ]:
df = auto_data.year == 20
auto_data[df].sort_values(by = 'price')

Еще лучше 2000 года с большим объемом двигателя и меньшим пробегом за 1490

In [ ]:
auto_data.corr(method='spearman')["price"].sort_values(ascending=False)

In [ ]:
auto_data_engineSize0 = auto_data['engineSize'].isin([0])
auto_data[auto_data_engineSize0]

https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes/discussion/209170 - engineSize = 0 - для электромобилей (ответ автора датасета)

In [ ]:
import plotly.express as px
model_count= auto_data
ax2= px.treemap(model_count,path=["model"],title="Popularity of the bought car models:")
ax2.show()

**Проверим датасет на дубликаты и удалим их**

In [ ]:
duplicated_df = auto_data.duplicated(keep = False)
auto_data[duplicated_df]

In [ ]:
auto_data=auto_data.drop_duplicates(keep="first")

In [ ]:
duplicated_df = auto_data.duplicated(keep = False)
auto_data[duplicated_df]

**Проверим номинальные признаки**

In [ ]:
auto_data.model.value_counts()

180, 200,220, 230 - это цифровые индексы, обозначающие объем двигателя, а не модель

In [ ]:
df = auto_data.model.isin(['200','180','220','230'])
auto_data[df]

более того, действительности они не соотвествуют<br>
что с ними делать?

In [ ]:
auto_data.loc[:,['model']] = auto_data['model'].replace(to_replace=['200','180','220','230'],value=np.NaN)
auto_data.dropna(how='any', axis=0, inplace=True)

In [ ]:
sns.countplot(auto_data["transmission"])

In [ ]:
auto_data['transmission'].value_counts()

In [ ]:
auto_data.loc[:,['transmission']] = auto_data['transmission'].replace(to_replace=['Other'],value=np.NaN)
auto_data.dropna(how='any', axis=0, inplace=True)

In [ ]:
sns.countplot(auto_data["fuelType"])

In [ ]:
auto_data['fuelType'].value_counts()

Можно подумать, что other относится к электроавтомобилям, но engineSize равно 0 только у одной модели

In [ ]:
df = auto_data.fuelType=='Other'
auto_data[df]

**Спросим эксперта**<br>

какие бывают виды топлива для авто<br>
кроме дизеля, бензина?
<br>
Семён 19:42<br>
Для авто только это<br>
Керосин - самолеты и ракеты<br>

*Мнение эксперта*<br>
*Поэтому выкинем их*

In [ ]:
auto_data.loc[:,['fuelType']] = auto_data['fuelType'].replace(to_replace=['Other'],value=np.NaN)
auto_data.dropna(how='any', axis=0, inplace=True)

In [ ]:
df = auto_data.fuelType=='Other'
auto_data[df]

In [ ]:
sns.countplot(y = auto_data["year"])

sns.pairplot(auto_data)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(auto_data.corr(method='spearman'), annot = True, linewidths=0.7, ax=ax, cmap="YlGnBu" )

Год и пробег с примерно одинаковой силой влияют на цену автомобиля. Прямая и обратная зависимость соответственно<br>
Налог в UK зависит от выбросов CO2 в воздух. Чем больше расход топлива, тем больше выброс. **<br>Почему тогда корреляция негативная?**

In [ ]:
plt.subplots(figsize=(12,7))
sns.lineplot(data=auto_data, x="mpg", y="tax")

In [ ]:
auto_data

**Закодируем строковые поля числовыми значениями**

***Как лучше закодировать? Label Encoder, dummy, OHE, frequency encoding***

In [ ]:
from sklearn.preprocessing import LabelEncoder
auto_data_label_encoder =  auto_data.copy(deep=True)

label_encoder = LabelEncoder()
label_encoder.fit(auto_data_label_encoder['model'])
label_encoder.transform(auto_data_label_encoder['model'])
auto_data_label_encoder['model'] = label_encoder.transform(auto_data_label_encoder['model'])

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(auto_data_label_encoder['transmission'])
label_encoder.transform(auto_data_label_encoder['transmission'])
auto_data_label_encoder['transmission'] = label_encoder.transform(auto_data_label_encoder['transmission'])

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(auto_data_label_encoder['fuelType'])
label_encoder.transform(auto_data_label_encoder['fuelType'])
auto_data_label_encoder['fuelType'] = label_encoder.transform(auto_data_label_encoder['fuelType'])
auto_data_label_encoder

In [ ]:
auto_data_label_encoder.head()

In [ ]:
auto_data_label_encoder.describe()

# Train test split

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

In [ ]:
X, Y = auto_data_label_encoder.drop(['price'], axis=1), auto_data_label_encoder['price']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10, shuffle=True)

In [ ]:
std_scaler = StandardScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled

# Линейная регрессия

In [ ]:
simple_LR = LinearRegression()

simple_LR.fit(X_train_scaled, Y_train)
simple_LR_predictions = simple_LR.predict(X_test_scaled)

simple_LR_MSE = metrics.mean_squared_error(Y_test, simple_LR_predictions)
simple_LR_MSE

In [ ]:
simple_LR_RMSE = metrics.mean_squared_error(Y_test, simple_LR_predictions, squared=False)
print('RMSE LinearRegression', simple_LR_RMSE) 

стандартное отклонение

In [ ]:
Y_test.std()

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(simple_LR_predictions, Y_test)
plt.xlabel('Predictions')
plt.ylabel('Target')

Судя по графику получилось не очень

In [ ]:
simple_LR.intercept_, simple_LR.coef_

In [ ]:
pd.DataFrame(simple_LR.coef_, auto_data_label_encoder.columns[1:], columns=['Coefficients'])

# Применим регуляризацию

In [ ]:
lasso_model = Lasso()

lasso_model.fit(X_train_scaled, Y_train)
predictions = lasso_model.predict(X_test_scaled)
metrics.mean_squared_error(Y_test, predictions, squared=False)


In [ ]:
pd.DataFrame(lasso_model.coef_, auto_data_label_encoder.columns[1:], columns=['Coefficients'])

In [ ]:
lasso_model = LassoCV(random_state=10)

lasso_model.fit(X_train_scaled, Y_train)
predictions = lasso_model.predict(X_test_scaled)

metrics.mean_squared_error(Y_test, predictions, squared=False)


Невероятно, но почти ничего не изменилось

In [ ]:
pd.DataFrame(lasso_model.coef_, auto_data_label_encoder.columns[1:], columns=['Coefficients'])

In [ ]:
ridge_model = Ridge()

ridge_model.fit(X_train_scaled, Y_train)
predictions = ridge_model.predict(X_test_scaled)

metrics.mean_squared_error(Y_test, predictions, squared=False)


In [ ]:
pd.DataFrame(ridge_model.coef_, auto_data_label_encoder.columns[1:], columns=['Coefficients'])

In [ ]:
ridge_model = RidgeCV()

ridge_model.fit(X_train_scaled, Y_train)
predictions = ridge_model.predict(X_test_scaled)

metrics.mean_squared_error(Y_test, predictions, squared=False)


In [ ]:
pd.DataFrame(ridge_model.coef_, auto_data_label_encoder.columns[1:], columns=['Coefficients'])

In [ ]:
alphas = np.linspace(1, 10, 20)
alphas

In [ ]:
lasso_model = LassoCV(random_state = 10, alphas=alphas)

lasso_model.fit(X_train_scaled, Y_train)
predictions = lasso_model.predict(X_test_scaled)

metrics.mean_squared_error(Y_test, predictions, squared=False)

In [ ]:

pd.DataFrame(lasso_model.coef_, auto_data_label_encoder.columns[1:], columns=['Coefficients'])

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr_model = RandomForestRegressor(random_state = 10, n_estimators = 100)

rfr_model.fit(X_train_scaled, Y_train)
predictions = rfr_model.predict(X_test_scaled)

print (f'RMSE = {round(metrics.mean_squared_error(Y_test, predictions, squared=False), 3)}')

pd.DataFrame(rfr_model.feature_importances_, X.columns, 
             columns=['Importance']).sort_values('Importance')

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(predictions, Y_test)
plt.xlabel('Predictions')
plt.ylabel('Target')

**Сравним разные модели**

In [ ]:
def regression_model(model):
    regressor = model
    regressor.fit(X_train_scaled, Y_train)

    score = regressor.score(X_test_scaled, Y_test)
    return regressor, score

In [ ]:

model_performance = pd.DataFrame(columns = ["Features", "Model", "Score"])
alphas = (1, 10, 20)
models_to_evaluate = [LinearRegression(), Ridge(), RidgeCV(alphas), Lasso(), LassoCV(), RandomForestRegressor(random_state = 10, n_estimators = 100)]

for model in models_to_evaluate:
    regressor, score = regression_model(model)
    model_performance = model_performance.append({"Features": "Linear","Model": model, "Score": score}, ignore_index=True)

model_performance

**Если не применять перемешивание при разделении данных на тестовые и тренировочные, результат значительно хуже**

In [ ]:
rfr_model.fit(X_train_scaled, Y_train)
score = rfr_model.score(X_test_scaled, Y_test)
score

# Модель без номинальных признаков

In [ ]:
auto_data_no_nominal_cols = auto_data_label_encoder.drop(['transmission','fuelType'], axis=1, inplace=False)

*если убрать model, будет совсем плохо*

In [ ]:
auto_data_no_nominal_cols.head()

In [ ]:
X, Y = auto_data_no_nominal_cols.drop(['price'], axis=1), auto_data_no_nominal_cols['price']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10, shuffle=True)

In [ ]:
std_scaler = StandardScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled

In [ ]:

rfr_model = RandomForestRegressor(random_state = 10, n_estimators = 100)

rfr_model.fit(X_train_scaled, Y_train)
predictions = rfr_model.predict(X_test_scaled)

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(predictions, Y_test)
plt.xlabel('Predictions')
plt.ylabel('Target')

In [ ]:
print (f'RMSE = {round(metrics.mean_squared_error(Y_test, predictions, squared=False), 3)}')

это ни к чему хорошему не привело, что и требовалось доказать

# Dummy Encoding

In [ ]:

auto_data_dummy = pd.concat([auto_data.drop(['fuelType','transmission','model'], axis =1, inplace=False),
                       pd.get_dummies(auto_data[['fuelType','transmission','model']])],axis=1)
auto_data_dummy

In [ ]:
X, Y = auto_data_dummy.drop(['price'], axis=1), auto_data_dummy['price']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10, shuffle=True)

In [ ]:
std_scaler = StandardScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled


In [ ]:
def regression_model(model):
    regressor = model
    regressor.fit(X_train_scaled, Y_train)

    score = regressor.score(X_test_scaled, Y_test)
    predictions = regressor.predict(X_test_scaled)
    RMSE = metrics.mean_squared_error(Y_test, predictions, squared=False)
    return regressor, score, RMSE

In [ ]:
model_performance = pd.DataFrame(columns = ["Features", "Model", "Score"])
alphas = (1, 10, 20)
models_to_evaluate = [LinearRegression(), Ridge(), RidgeCV(alphas), Lasso(), LassoCV(), RandomForestRegressor()]

for model in models_to_evaluate:
    regressor, score, RMSE = regression_model(model)
    model_performance = model_performance.append({"Features": "Linear","Model": model, "Score": score, "RMSE": RMSE}, ignore_index=True)

model_performance

In [ ]:
rfr_model = RandomForestRegressor(random_state = 10, n_estimators = 100)

rfr_model.fit(X_train_scaled, Y_train)
predictions = rfr_model.predict(X_test_scaled)

In [ ]:
print('RMSE:', metrics.mean_squared_error(Y_test, predictions, squared=False))

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(predictions, Y_test)
plt.xlabel('Predictions')
plt.ylabel('Target')

In [ ]:
pd.DataFrame(rfr_model.feature_importances_, X.columns, 
             columns=['Importance']).sort_values('Importance')